# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [5]:
# Load API Credentials
with open('/Users/purvikansara/.secret/yelp_api.json') as f:
    creds = json.load(f)
creds.keys()

dict_keys(['Client-ID', 'API Key'])

In [6]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(creds['API Key'],timeout_s=5.0)

### Define Search Terms and File Paths

In [7]:
# set our API call parameters and filename before the first call
location  = 'Seattle, WA 98122'
term = 'pizza'

In [9]:
## Specify fodler for saving data

FOLDER = 'Data/'
os.makedirs(FOLDER,exist_ok=True)
# Specifying JSON_FILE filename (can include a folder)
# method1
#JSON_FILE = FOLDER+'Seattle pizza.json'

#method2
location.split(',')[0]
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

In [10]:
JSON_FILE


'Data/Seattle-pizza.json'

### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [12]:
# use our yelp_api variable's search_query method to perform our API call
results =yelp.search_query(term=term,location=location)
type(results)

dict

In [13]:
results

{'businesses': [{'id': 't-Z9bvmlgUyDtomGmttrUQ',
   'alias': 'guerilla-pizza-kitchen-seattle',
   'name': 'Guerilla Pizza Kitchen',
   'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEjc9J_4Tb1PtAb71UA/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/guerilla-pizza-kitchen-seattle?adjust_creative=r8Qlusnw5U_OXTmzDH90Ug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=r8Qlusnw5U_OXTmzDH90Ug',
   'review_count': 1,
   'categories': [{'alias': 'pizza', 'title': 'Pizza'},
    {'alias': 'popuprestaurants', 'title': 'Pop-Up Restaurants'}],
   'rating': 5.0,
   'coordinates': {'latitude': 47.608127, 'longitude': -122.302435},
   'transactions': [],
   'location': {'address1': '2300 E Cherry St',
    'address2': '',
    'address3': None,
    'city': 'Seattle',
    'zip_code': '98122',
    'country': 'US',
    'state': 'WA',
    'display_address': ['2300 E Cherry St', 'Seattle, WA 98122']},
   'phone': '',
   'display_phone': '',
   'distance': 7

In [14]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [15]:

results['businesses'][0]

{'id': 't-Z9bvmlgUyDtomGmttrUQ',
 'alias': 'guerilla-pizza-kitchen-seattle',
 'name': 'Guerilla Pizza Kitchen',
 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEjc9J_4Tb1PtAb71UA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/guerilla-pizza-kitchen-seattle?adjust_creative=r8Qlusnw5U_OXTmzDH90Ug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=r8Qlusnw5U_OXTmzDH90Ug',
 'review_count': 1,
 'categories': [{'alias': 'pizza', 'title': 'Pizza'},
  {'alias': 'popuprestaurants', 'title': 'Pop-Up Restaurants'}],
 'rating': 5.0,
 'coordinates': {'latitude': 47.608127, 'longitude': -122.302435},
 'transactions': [],
 'location': {'address1': '2300 E Cherry St',
  'address2': '',
  'address3': None,
  'city': 'Seattle',
  'zip_code': '98122',
  'country': 'US',
  'state': 'WA',
  'display_address': ['2300 E Cherry St', 'Seattle, WA 98122']},
 'phone': '',
 'display_phone': '',
 'distance': 749.1731129459927}

In [23]:
pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 47.608127, 'longitude': -122.302435}",[],"{'address1': '2300 E Cherry St', 'address2': '...",,,749.173113,NaN
1,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,925,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.615379179632, 'longitude': -12...",[delivery],"{'address1': '700 E Pine St', 'address2': '', ...",+12063226444,(206) 322-6444,2080.359405,$
2,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,30,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,"{'latitude': 47.61849, 'longitude': -122.31664}",[],"{'address1': '1830 12th Ave', 'address2': '', ...",+12064031809,(206) 403-1809,1686.862176,NaN
3,ugTsEtjvwRhteac_6JcuPw,italian-family-pizza-seattle,Italian Family Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/SgiSrJ...,False,https://www.yelp.com/biz/italian-family-pizza-...,972,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 47.60937, 'longitude': -122.32546}","[pickup, delivery]","{'address1': '1028 Madison St', 'address2': No...",+12065380040,(206) 538-0040,2271.307105,$$
4,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media1.fl.yelpcdn.com/bphoto/9tK83_...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,349,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.5976491915013, 'longitude': -1...",[delivery],"{'address1': '525 Rainier Ave S', 'address2': ...",+12063295133,(206) 329-5133,2166.834011,$$
5,cAFXng3Gr1cBCFkP1VIxng,the-independent-pizzeria-seattle-2,The Independent Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/4gIuis...,False,https://www.yelp.com/biz/the-independent-pizze...,179,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 47.636044, 'longitude': -122.277377}",[delivery],"{'address1': '4235 E Madison St', 'address2': ...",+12068606110,(206) 860-6110,2897.252844,$$
6,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,86,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.624577, 'longitude': -122.325577}","[pickup, delivery]","{'address1': '601 Summit Ave E', 'address2': '...",,,2578.701393,NaN
7,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,330,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 47.6146934, 'longitude': -122.312...",[delivery],"{'address1': '1546 15th Ave', 'address2': '', ...",+12068388081,(206) 838-8081,1292.526796,$$
8,7hfyyyVKXOQMOMWTkFrvNg,boca-argentine-bakery-and-pizzeria-seattle,Boca Argentine Bakery & Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/5MPTtV...,False,https://www.yelp.com/biz/boca-argentine-bakery...,16,"[{'alias': 'argentine', 'title': 'Argentine'},...",4.5,"{'latitude': 47.62278, 'longitude': -122.32064}",[],"{'address1': '426 Broadway E', 'address2': '',...",,,2152.679112,NaN
9,IrohtoYjnAR_vc6w6CRCxA,moto-seattle,Moto,https://s3-media2.fl.yelpcdn.com/bphoto/FBmvjy...,False,https://www.yelp.com/biz/moto-seattle?adjust_c...,110,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 47.56207, 'longitude': -122.38509}",[delivery],"{'address1': '4526 42nd Ave SW', 'address2': '...",+12064208880,(206) 420-8880,8784.075722,NaN


In [16]:
results['total']

844

In [17]:
results['region']

{'center': {'longitude': -122.29568481445312, 'latitude': 47.61309534151991}}

In [18]:
results_per_page = len(results['businesses'])

In [19]:
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [21]:
## How many results total?
# Use math.ceil to round up for the total number of pages of results.
import math
n_pages= math.ceil(results['total']/results_per_page)
n_pages

43

In [27]:
# lets save the data

# Check if Json file exists and create it if it does not

if os.path.isfile(JSON_FILE)==False:
    # file does not exist so print message and create empty file
    print(" The file does not exist. Creating empty file.")
    with open(JSON_FILE,'w') as f:
        json.dump(results['businesses'],f)
# if file exists
else:
    print("File already exists")



 The file does not exist. Creating empty file.


Lets use Python library tqdm to create simple progress bars

In [28]:
for i in tqdm_notebook( range(1,n_pages+1)):
        ## The block of code we want to TRY to run
        
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE)as f:
            prev_results = json.load(f)
        
        ## save number of results for to use as offset
        
        n_results = len(prev_results)
        
        ## use n_results as the OFFSET 
        results = yelp.search_query(term = term,location=location,offset=n_results)

        ## append new results and save to file
        prev_results.extend(results['businesses'])
        with open(JSON_FILE,'w') as f:
            json.dump(prev_results,f)
        
            

        


  0%|          | 0/43 [00:00<?, ?it/s]

## Open the Final JSON File with Pandas

In [29]:
df = pd.read_json(JSON_FILE)
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 47.608127, 'longitude': -122.302435}",[],"{'address1': '2300 E Cherry St', 'address2': '...",,,749.173113,NaN
1,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,925,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.615379179632, 'longitude': -12...",[delivery],"{'address1': '700 E Pine St', 'address2': '', ...",+12063226444,(206) 322-6444,2080.359405,$
2,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,30,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,"{'latitude': 47.61849, 'longitude': -122.31664}",[],"{'address1': '1830 12th Ave', 'address2': '', ...",+12064031809,(206) 403-1809,1686.862176,NaN
3,ugTsEtjvwRhteac_6JcuPw,italian-family-pizza-seattle,Italian Family Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/SgiSrJ...,False,https://www.yelp.com/biz/italian-family-pizza-...,972,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 47.60937, 'longitude': -122.32546}","[pickup, delivery]","{'address1': '1028 Madison St', 'address2': No...",+12065380040,(206) 538-0040,2271.307105,$$
4,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media1.fl.yelpcdn.com/bphoto/9tK83_...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,349,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.5976491915013, 'longitude': -1...",[delivery],"{'address1': '525 Rainier Ave S', 'address2': ...",+12063295133,(206) 329-5133,2166.834011,$$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,tyjUpcq1IYfgrss-HO9hog,panera-bread-bellevue,Panera Bread,https://s3-media2.fl.yelpcdn.com/bphoto/XyC46x...,False,https://www.yelp.com/biz/panera-bread-bellevue...,218,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",2.5,"{'latitude': 47.6204515, 'longitude': -122.199...","[delivery, pickup]","{'address1': '1100 Bellevue Way NE', 'address2...",+14254532600,(425) 453-2600,7246.971006,$
840,8S3qU0TGMJDyj4KHBmxlaw,pho-tai-kirkland,Pho Tai,https://s3-media3.fl.yelpcdn.com/bphoto/tnvGkA...,False,https://www.yelp.com/biz/pho-tai-kirkland?adju...,187,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",3.5,"{'latitude': 47.676215, 'longitude': -122.205084}","[pickup, delivery]","{'address1': '147 Park Ln', 'address2': '', 'a...",+14255765407,(425) 576-5407,9765.353049,$$
841,H_gsS9J920u_hK7jekobqw,suite-bellevue-4,Suite,https://s3-media1.fl.yelpcdn.com/bphoto/jdYMfv...,False,https://www.yelp.com/biz/suite-bellevue-4?adju...,221,"[{'alias': 'lounges', 'title': 'Lounges'}, {'a...",3.0,"{'latitude': 47.6179959197781, 'longitude': -1...",[delivery],"{'address1': '10500 NE 8th St', 'address2': 'S...",+14256796951,(425) 679-6951,7158.825860,$$
842,rcDbUcs83gvoRxhaTmFfnQ,cypress-lounge-and-wine-bar-bellevue,Cypress Lounge & Wine Bar,https://s3-media2.fl.yelpcdn.com/bphoto/DsFhIb...,False,https://www.yelp.com/biz/cypress-lounge-and-wi...,64,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",3.5,"{'latitude': 47.6157033283809, 'longitude': -1...",[],"{'address1': '600 Bellevue Way NE', 'address2'...",+14256381000,(425) 638-1000,7123.208902,$$


In [30]:
# check for any duplicates
df.duplicated(subset='id').sum()

0

In [31]:
# remove duplicates if any
df.drop_duplicates(subset='id',inplace=True)

In [32]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Seattle-pizza.csv.gz'

In [33]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file,compression='gzip',index=False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [34]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 823,000 Bytes
CSV.GZ FILE: 119,112 Bytes
the csv.gz is 6.909463362213715 times smaller!


## Next Class: Processing the Results and Mapping 